In [ ]:
Libraries Used
pandas
sklearn
datasets
transformers
torch

In [3]:
#Ignore this one, unless the cell below gives error, it's an alternative
#Import Dataset as array of chunks
import pandas as pd

input_file = '/home/charlie/Desktop/Data_Subset/Masterfile/subset_1.csv'
chunk_size=10000
try:
    #Load dataset in chunks to prevent overload
    df= pd.read_csv(input_file,chunksize=chunk_size)
    for i, chunk in enumerate(df):
        print(f"Processing Chunk {i+1}")
    
except Exception as e:
    print(f"CRITICAL ERROR: {e}")
    
print("CELL COMPLETE")

Processing Chunk 1
Processing Chunk 2
Processing Chunk 3
Processing Chunk 4
Processing Chunk 5
Processing Chunk 6
Processing Chunk 7
Processing Chunk 8
Processing Chunk 9
Processing Chunk 10
Processing Chunk 11
Processing Chunk 12
Processing Chunk 13
Processing Chunk 14
Processing Chunk 15
Processing Chunk 16
Processing Chunk 17
Processing Chunk 18
Processing Chunk 19
Processing Chunk 20
Processing Chunk 21
Processing Chunk 22
Processing Chunk 23
Processing Chunk 24
Processing Chunk 25
Processing Chunk 26
Processing Chunk 27
Processing Chunk 28
Processing Chunk 29
Processing Chunk 30
Processing Chunk 31
Processing Chunk 32
Processing Chunk 33
Processing Chunk 34
Processing Chunk 35
Processing Chunk 36
Processing Chunk 37
Processing Chunk 38
Processing Chunk 39
Processing Chunk 40
Processing Chunk 41
Processing Chunk 42
Processing Chunk 43
Processing Chunk 44
Processing Chunk 45
Processing Chunk 46
Processing Chunk 47
Processing Chunk 48
Processing Chunk 49
Processing Chunk 50
Processin

In [2]:
#Alternatively if it can load all dataframes without crashing
import pandas as pd

input_file = '/home/charlie/Desktop/Data_Subset/Masterfile/subset_5.csv'
chunk_size = 10000

try:
    df = pd.concat(pd.read_csv(input_file, chunksize=chunk_size))
    print(df.shape)
    print(df['label'].value_counts())
    print("Successfully loaded all dataframes")
except Exception as e:
    print(f"CRITICAL ERROR: {e}")

print("Cell Completed")

(24000000, 23)
label
Benign       12000000
Malicious    12000000
Name: count, dtype: int64
Successfully loaded all dataframes
Cell Completed


In [3]:
#splitting data
import sklearn
from sklearn.model_selection import train_test_split
#below specified 0.3 of df is seperated to test, with a distrubtion of label column
train, test = train_test_split(df,test_size=0.3,stratify=df['label'])
#creates validation set by taking 1/3 of test
test, validation= train_test_split(test, test_size=1/3,stratify=test['label'])

print("(rows,columns)\n")
print(f"Data Frame:{df.shape}")
print(f"Training data:{train.shape}")
print(f"Test data:{test.shape}")
print(f"validation data:{validation.shape}")
print("Cell Completed")

(rows,columns)

Data Frame:(24000000, 23)
Training data:(16800000, 23)
Test data:(4800000, 23)
validation data:(2400000, 23)
Cell Completed


In [7]:
#WORKS FOR SMALLER DATASETS BUT CRASHES WITH LARGER USE CELL BELOW
#convert dataframes in dataset dictionary objects 
from datasets import Dataset, DatasetDict
try:
    dataset= DatasetDict(
        { "train": Dataset.from_pandas(train,preserve_index=False),
          "test": Dataset.from_pandas(test,preserve_index=False),
          "validation": Dataset.from_pandas(validation,preserve_index=False)
        }
    )

except Exception as e:
    print(f"CRITICAL ERROR: {e}")

dataset
print("Cell Completed")

CRITICAL ERROR: from_pandas() got an unexpected keyword argument 'streaming'
Cell Completed


In [4]:
#convert dataframes in dataset dictionary objects, uses chunking to work with large datasets
from datasets import Dataset, DatasetDict, load_dataset, concatenate_datasets

chunk_size = 10000
try:
    def df_chunking(df, chunk_size=chunk_size):
        for start in range(0, len(df),chunk_size):
            yield Dataset.from_pandas(df.iloc[start:start + chunk_size],preserve_index=True)

    #make each dataframe split a chunk 
    train_chunks= list(df_chunking(train))
    test_chunks= list(df_chunking(test))
    validation_chunks= list(df_chunking(validation))

    #Concatenates each dataframe chunk into datasets
    train_dataset = concatenate_datasets(train_chunks)
    test_dataset = concatenate_datasets(test_chunks)
    validation_dataset = concatenate_datasets(validation_chunks)



    #makes data dictionaries
    dataset= DatasetDict(
        { "train": train_dataset,
          "test": test_dataset,
          "validation": validation_dataset
        }
    )

except Exception as e:
    print(f"CRITICAL ERROR: {e}")

dataset
print("Cell Completed")

/home/charlie/anaconda3/envs/FYP_Env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cell Completed


In [5]:
print(f"dataset:{dataset}")

dataset:DatasetDict({
    train: Dataset({
        features: ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents', 'label', 'detailed-label', '__index_level_0__'],
        num_rows: 16800000
    })
    test: Dataset({
        features: ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents', 'label', 'detailed-label', '__index_level_0__'],
        num_rows: 4800000
    })
    validation: Dataset({
        features: ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local

In [17]:
#Tokenisation
from transformers import AutoTokenizer
input_sample="1533042911.474174,C5JLGOoxIw2dBZt47,192.168.100.113,123,81.2.254.224,123,udp,,0.00549,48.0,48.0,SF,,,0,Dd,1,76,1,76,(empty),Benign,-"
model_checkpoint="distilbert-base-uncased"

distilbert_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
distilber_token = distilbert_tokenizer(input_sample)

distilbert_tokenizer



DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [18]:
#testing tokenizer with input_sample as its log file data and no natural language

tokens = distilbert_tokenizer.tokenize(input_sample)
print(tokens)


['153', '##30', '##42', '##9', '##11', '.', '47', '##41', '##7', '##4', ',', 'c', '##5', '##j', '##l', '##go', '##ox', '##i', '##w', '##2', '##db', '##z', '##t', '##47', ',', '192', '.', '168', '.', '100', '.', '113', ',', '123', ',', '81', '.', '2', '.', '254', '.', '224', ',', '123', ',', 'ud', '##p', ',', ',', '0', '.', '00', '##54', '##9', ',', '48', '.', '0', ',', '48', '.', '0', ',', 'sf', ',', ',', ',', '0', ',', 'dd', ',', '1', ',', '76', ',', '1', ',', '76', ',', '(', 'empty', ')', ',', 'benign', ',', '-']


In [7]:
# Import necessary modules
from transformers import AutoTokenizer, AutoModel

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")  
model = AutoModel.from_pretrained("distilbert-base-uncased") 

# new tokens
new_token = [
    "1533042911.474174", "C5JLGOoxIw2dBZt47", "192.168.100.113,123",
    "81.2.254.224,123", "udp", "", "0.00549,48.0", "48.0", "SF", "", "",
    "0", "Dd", "1", "76", "1", "76", "(empty)", "Benign", "-", 
    "1536241151.35095", "C9m6XdPw0pRf5lNa1", "192.168.100.111", 
    "18344,156.212.84.219", "52869", "tcp", "", "4e-06", "0.0", "0.0", 
    "S0", "", "", "0", "S", "2", "80", "0", "0", "-", "Malicious", 
    "DDoS", "1536232599.053273", "C0lARQ2hM1UUe4aSrg", "192.168.100.111", 
    "17832", "156.112.51.76", "37215", "tcp", "", "1e-06", "0.0", "0.0", 
    "S0", "", "", "0", "S", "2", "80", "0", "0", "-", "Malicious", 
    "Okiru", "1536229808.84214", "CTL3WP1XmST9PFud2", "192.168.100.111", 
    "17576", "53.178.3.247", "8081", "tcp", "", "1e-06", "0.0", "0.0", 
    "S0", "", "", "0", "S", "2", "80", "0", "0", "-", "Malicious", 
    "PartOfAHorizontalPortScan", "ts", "uid", "id.orig_h", "id.orig_p", 
    "id.resp_h", "id.resp_p", "proto", "service", "duration", 
    "orig_bytes", "resp_bytes", "conn_state", "local_orig", "local_resp", 
    "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", 
    "resp_ip_bytes", "tunnel_parents", "label", "detailed-label"
]

# Add tokens to tokenizer
num_added_tokens = tokenizer.add_tokens(new_token)
print(f"Added {num_added_tokens} new tokens.")

# Resize the token embeddings
model.resize_token_embeddings(len(tokenizer))


2025-01-08 17:02:50.768408: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-08 17:02:50.876031: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-08 17:02:50.876067: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-08 17:02:50.890126: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-08 17:02:50.927367: I tensorflow/core/platform/cpu_feature_guar

Added 65 new tokens.


Embedding(30575, 768, padding_idx=0)

In [9]:
# Example input sample (log file data)
input_sample="1533042911.474174,C5JLGOoxIw2dBZt47,192.168.100.113,123,81.2.254.224,123,udp,,0.00549,48.0,48.0,SF,,,0,Dd,1,76,1,76,(empty),Benign,-"


# Tokenize the input sample
tokens = tokenizer.tokenize(input_sample)
print(tokens)


['1533042911.474174', ',', 'c5jlgooxiw2dbzt47', ',', '192.168.100.113,123', ',', '81.2.254.224,123', ',', 'udp', ',', ',', '0.00549,48.0', ',', '48.0', ',', 'sf', ',', ',', ',', '0', ',', 'dd', ',', '1', ',', '76', ',', '1', ',', '76', ',', '(empty)', ',', 'benign', ',', '-']
